In [1]:
import pandas as pd
import numpy as np
import warnings
import pyodbc
import random
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from collections import Counter
import multiprocessing as mp
import ray
import datetime

warnings.filterwarnings(action = 'ignore')

2023-12-07 10:45:32,984	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.3 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


#### 유전 알고리즘 활용한 페어 트레이딩_섹터 확장_v1

* v8
 - 비중 제한 없애기
 - child 생성 시 볼린저 밴드 계산 기간과 비중 모두 BLX-alpha crossover 적용
 - mutation 시 랜덤넘버 생성을 통해 각 파라미터에 더하기
 - 변이 이후 수선 기능 추가. 1) 롱/숏 비중 각 1로 맞추기
 - train set 기간 0.7에 대해 시뮬레이션 해보기
 - 정상성 테스트 제외: rolling base 기반 표준화를 사용하기 때문

In [2]:
conn_pcor = pyodbc.connect('driver={Oracle in OraClient18Home1};dbq=PCOR;uid=EF0SEL;pwd=EF0SEL#076')
conn_quant = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=quant;uid=index;pwd=samsung@00')
conn_wisefn = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=wisefn;uid=index;pwd=samsung@00')

In [3]:
os.chdir("..")
os.chdir("..")

os.chdir('00_data')

#df_const = pd.read_json('230927_index_constituent.json')
df_prc_raw = pd.read_json('230927_stk_prc_daily.json')
#df_turnover = pd.read_json('230927_stk_turnover.json')
#df_sector = pd.read_json('230927_stk_sector.json')
df_mktcap = pd.read_json('230927_stk_mktcap.json')
#df_turnover_daily = pd.read_json('231011_stk_turnover_daily.json')

os.chdir("..")
os.chdir('02_Trading Strategies')
os.chdir('231026_유전 알고리즘을 활용한 페어트레이딩')

In [4]:
sector_code = 'IKS013'

In [5]:
sql_dt = f'''
        SELECT TRD_DT, CLOSE_PRC
        FROM TS_IDX_DAILY
        WHERE 1=1
        AND SEC_CD = '{sector_code}'
        AND TRD_DT > '20091231'
        ORDER BY TRD_DT ASC
'''

df_dt = pd.read_sql(sql_dt, conn_wisefn)

In [6]:
df_dt['TRD_DT'] =  pd.to_datetime(df_dt['TRD_DT'])
df_dt['M'] = df_dt['TRD_DT'].dt.to_period('M')
dt_m = [max(df_dt[df_dt['M']==p]['TRD_DT']) for p in df_dt['M'].unique()]
dt_m = [int(x.strftime('%Y%m%d')) for x in dt_m]

In [7]:
def objective_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = np.log(df) * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]
    
    ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]
        
        ret = prc_2/prc_1 - 1
        ret = (ret * params_wt).sum()
        ret_list.append(ret)
        cum_ret *= (1+ret)
    
    return cum_ret

In [67]:
def generate_weight(length):
    
    pos_len = random.randint(1,length - 1)
    neg_len = length - pos_len
    
    pos_wt = []
    for i in range(pos_len):
        p = random.uniform(0,1)
        pos_wt.append(p)
    
    pos_wt = [x/sum(pos_wt) for x in pos_wt]
    
    neg_wt = []
    for i in range(neg_len):
        n = random.uniform(0,1)
        neg_wt.append(n)
    
    neg_wt = [-x/sum(neg_wt) for x in neg_wt]
    
    wt = pos_wt + neg_wt
    
    random.shuffle(wt)
    
    return wt  
        

def generate_params(length):
    
    params_period = random.randint(20,20*18)
    params_init = random.uniform(-4,0)
    params_exit = random.uniform(0,4)
    params_wt = generate_weight(length)
    
    params = [params_period, params_init, params_exit, *params_wt]
            
    return params
    

In [68]:
def generate_pop(size, length):
    population = []
    
    for i in range(size):
        
        params = generate_params(length)
        population.append(params)
        
    return population

In [69]:
def compute_performance(df, population):
    
    obj_list = []
    for individual in population:
        obj = objective_func(df, individual)
        obj_list.append([individual,obj])
    
    pop_sorted = sorted(obj_list, key=lambda x: x[1], reverse=True)
    
    return pop_sorted

In [70]:
def select_survivors(population_sorted, best_sample, lucky_few, length):
    
    next_gen = []
    
    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_gen.append(population_sorted[i][0])
    
    lucky_index = np.random.choice(list(range(len(population_sorted))), lucky_few, replace=False)
    for i in lucky_index:
        next_gen.append(population_sorted[i][0])
    
    next_gen_2 = []
    for item in next_gen:
        if item not in next_gen_2:
            next_gen_2.append(item)
    
    
    while len(next_gen_2) < best_sample + lucky_few:
        next_gen_2.append(generate_params(length))
        
    random.shuffle(next_gen_2)
    
    return next_gen_2

In [71]:
def create_child(param1, param2, alpha):
    
    child = param1.copy()
    
    for i in range(len(child)):
        
        dist = abs(param1[i]-param2[i])
        
        l = min(param1[i],param2[i]) - alpha * dist
        u = max(param1[i],param2[i]) + alpha * dist

        child[i] = l + random.random() * (u-l)
    
    child[0] = int(np.round(child[0]))
    child[0] = min(20*18, child[0])
    child[0] = max(20, child[0])
    
    return child


def create_children(df, population, n_child, alpha, r_param):
    
    k = 0
    children = []

    fitness = [objective_func(df, x) for x in population]
    roulette = [x - min(fitness) + (max(fitness) - min(fitness))/(r_param - 1) for x in fitness]
    #fitness_2 = [x if x >= 0 else 0 for x in fitness]
    
    prob = [x/sum(roulette) for x in roulette]
    
    while k < n_child:
        try:
            sample = np.random.choice(list(range(len(prob))), 2, replace=True, p = prob)
        except:
            sample = np.random.choice(list(range(len(prob))), 2, replace=True)
        parents = [population[sample[0]], population[sample[1]]]
        
        child = create_child(parents[0], parents[1], alpha)
        children.append(child)
        
        k += 1

    return children

In [72]:
def mutation1(param, prob):
    
    r1 = random.random()
    
    param_mutate = param.copy()
    
    if r1 < prob:
        m1 = np.random.normal(0, 20)
        new_period = m1 + param[0]
        new_period = max(20, new_period)
        new_period = min(20*18, new_period)
        
        param_mutate[0] = int(np.round(new_period))
        
        m2 = np.random.normal(0,0.02,len(param) - 1)
        param_mutate[1:] = [x+y for (x,y) in zip(param_mutate[1:], m2)]
    
    return param_mutate


################################################
def mutate_pop(population, prob1):
    
    for i in range(len(population)):
        population[i] = mutation1(population[i], prob1)
    
    return population

In [73]:
def fix_1(population, length):
    
    # 수선: 비중 조절
    
    fixed_pop = []
    
    for child in population:
    
        child_wt = child[3:]
        child_pos = [x if x >= 0 else 0 for x in child_wt]
        child_neg = [x if x < 0 else 0 for x in child_wt]

        try:
            child_pos_adj = [x/sum(child_pos) for x in child_pos]
            child_neg_adj = [-x/sum(child_neg) for x in child_neg]
            child_wt_adj = [x+y for (x,y) in zip(child_pos_adj, child_neg_adj)]
            adj_child = child[:3] + child_wt_adj
        
        except:
            new_wt = generate_params(length)[3:]
            adj_child = child[:3] + new_wt
        
        fixed_pop.append(adj_child)
    
    return fixed_pop

In [74]:
def simulation_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = np.log(df) * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]

    ret_list = []
    long_ret_list = []
    short_ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]

        
        ret = prc_2/prc_1 - 1
        ret_ls = (ret * params_wt).sum()
        ret_list.append(ret_ls)        
        cum_ret *= (1+ret_ls)
        
        long_wt = [x if x >= 0 else 0 for x in params_wt]
        long_ret = (ret * long_wt).sum()
        long_ret_list.append(long_ret)
        
        short_wt = [x if x < 0 else 0 for x in params_wt]
        short_ret = (ret * short_wt).sum()
        short_ret_list.append(short_ret)
        
        
    
    return cum_ret, init_dt, exit_dt, ret_list, long_ret_list, short_ret_list

In [75]:
sector_code_list = ['IKS005', 'IKS006', 'IKS007', 'IKS008', 'IKS009', 'IKS010', 'IKS011', 'IKS012', 
                    'IKS013', 'IKS014', 'IKS015', 'IKS016', 'IKS017', 'IKS018', 'IKS019', 'IKS020', 
                    'IKS024', 'IKS025', 'IKS026']

In [76]:
sector_name_list = ['음식료품', '섬유의복', '종이목재', '화학', '의약품', '비금속광물', '철강금속', 
                    '기계', '전기전자', '의료정밀', '운수장비', '유통업', '전기가스업', '건설업', 
                    '운수창고업', '통신업', '증권', '보험업', '서비스업']

In [77]:
dict_sector_name = dict(zip(sector_code_list, sector_name_list))

In [78]:
i = 0.70
cap_hurdle = 200 * 1000000000
max_stk_cnt = 10

base_d = dt_m[int(np.round(len(dt_m)*i,0))]
train_begin_dt = dt_m[0]
train_end_dt = base_d

cap_filter = df_mktcap.copy()
cap_filter = cap_filter[cap_filter['BASE_D'] == base_d]
cap_filter = cap_filter[cap_filter['MKTCAP'] > cap_hurdle]
cap_filter.sort_values('MKTCAP', ascending = False, inplace = True)

dict_train = {}
dict_test = {}
dict_stk = {}

for sector_code in sector_code_list:
    
    sql_const = f'''
                SELECT TRD_DT, CONCAT('A',STK_CD) COMP_CD, STK_NM_KOR COMP_NM
                FROM TS_STK_ISSUE
                WHERE 1=1
                AND KS200_TYP = 1
                AND TRD_DT = '{base_d}'
                AND KSC_CD = '{sector_code}'
    '''

    df_const = pd.read_sql(sql_const, conn_wisefn)

    df_train = df_prc_raw.copy()
    df_train = df_train[df_train['TRD_DT'] >= train_begin_dt]
    df_train = df_train[df_train['TRD_DT'] <= train_end_dt]
    df_train = df_train[df_train['COMP_CD'].isin(df_const['COMP_CD'])]
    df_train = df_train.sort_values(by = ['TRD_DT','COMP_CD'])
    #stk_list = sorted(list(set(df_train['COMP_CD'])))


    df_train = df_train.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_train = df_train.droplevel(axis = 1, level = 0)
    #df_train = df_train[stk_list]
    df_train.dropna(axis=1, inplace=True)
    stk_list = sorted(list(df_train.columns))
    
    # 시가총액 필터링
    stk_list = cap_filter[cap_filter['COMP_CD'].isin(stk_list)]['COMP_CD'][:max_stk_cnt].to_list()

    df_train = df_train[stk_list]
    
    df_test = df_prc_raw.copy()
    df_test = df_test[df_test['TRD_DT'] >= train_end_dt]
    #df_test = df_prc[df_prc['TRD_DT'] <= train_end_dt]
    df_test = df_test[df_test['COMP_CD'].isin(df_const['COMP_CD'])]
    df_test = df_test.sort_values(by = ['TRD_DT','COMP_CD'])

    df_test = df_test.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_test = df_test.droplevel(axis = 1, level = 0)
    df_test = df_test[stk_list]
    
    dict_train[sector_code] = df_train
    dict_test[sector_code] = df_test
    dict_stk[sector_code] = stk_list

#### 병렬처리

In [54]:
ray.init(num_cpus=8)

2023-12-07 13:32:17,991	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.8
Ray version:,2.8.0
Dashboard:,http://127.0.0.1:8265


In [79]:
filtered_sector = []
for sector_code in dict_stk.keys():
    if len(dict_stk[sector_code]) < 2:
        pass
    else:
        filtered_sector.append(sector_code)

In [80]:
dict_train_2 = {k:dict_train[k] for k in filtered_sector if k in dict_train}
dict_test_2 = {k:dict_test[k] for k in filtered_sector if k in dict_test}
dict_stk_2 = {k:dict_stk[k] for k in filtered_sector if k in dict_stk}
dict_sector_name_2 = {k:dict_sector_name[k] for k in filtered_sector if k in dict_sector_name}

In [81]:
@ray.remote
def get_optimized_params(sector_code, dict_train, dict_test, dict_stk, dict_sector_name):
    
    print(f'섹터: {dict_sector_name[sector_code]}')
    #print(' ')
    
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    stk_list = dict_stk[sector_code]
    
    
    #print(f'  Train set: {train_begin_dt} ~ {train_end_dt}')
    #print(f'  Test set: {train_end_dt} ~ {max(df_test.index)}')
    #print(' ')
    #print(f'  유전 알고리즘 시작')
    n_generation = 200
    population = 1000
    best_sample = int(population/4)
    lucky_few = int(population/4)
    prob1 = 0.01
    #prob2 = 0.01
    alpha = 0.10
    r_param = 4
    param_length = len(stk_list)

    pop = generate_pop(size = population, length = param_length)

    best_gene = []
    best_perf = []

    g = 0
    
    cut = int(np.round(len(df_train)/3))

    while True:
        
        r = random.randint(1,len(df_train) - cut)
        
        df_train_sub = df_train.iloc[r:r+cut]

        pop_sorted = compute_performance(df_train_sub, pop)

        survivors = select_survivors(pop_sorted, best_sample, lucky_few, param_length)

        children = create_children(df_train_sub, survivors, population, alpha, r_param)

        new_generation = mutate_pop(children, prob1)

        if len(new_generation) < population:
            #print(f'  세대수 부족: {len(new_generation)}')
            break

        fixed_generation = fix_1(new_generation, param_length)

        pop = fixed_generation
        best_gene.append(pop_sorted[0][0])
        best_perf.append(pop_sorted[0][1])


        #print(f'====== {g}th generation ends ======')
        #print(pop_sorted[0])


        pop_wt = [x[0][3:] for x in pop_sorted]
        dist_wt = [sum([(x-y)**2 for (x,y) in zip(pop_wt[0],z)]) for z in pop_wt]
        hurdle = 0.025
        dist_wt_hurdle = [1 for x in dist_wt if x < hurdle]

        #print(f'    최다 출현 개체수: {sum(dist_wt_hurdle)}')
        if sum(dist_wt_hurdle) > population * 0.7:
            break

        g += 1

        if g > n_generation:
            break

        print(' ')

    #print(f'  유전 알고리즘 종료')
    #print(' ')
    
    return pop_sorted[0][0], g

In [82]:
futures = [get_optimized_params.remote(sector_code, dict_train_2, dict_test_2, dict_stk_2, dict_sector_name_2) for sector_code in filtered_sector]

(get_optimized_params pid=20408) 섹터: 의약품
(get_optimized_params pid=33928) 섹터: 음식료품
(get_optimized_params pid=32148) 섹터: 화학
(get_optimized_params pid=13420) 섹터: 철강금속
(get_optimized_params pid=33200) 섹터: 섬유의복
(get_optimized_params pid=21360) 섹터: 비금속광물
(get_optimized_params pid=28052) 섹터: 기계
(get_optimized_params pid=28184) 섹터: 전기전자


In [83]:
t1 = datetime.datetime.today()
optimized_params = ray.get(futures)

t2 = datetime.datetime.today()

lt = (t2 - t1).seconds

(get_optimized_params pid=20408)  
(get_optimized_params pid=32148)   [repeated 8x across cluster]
(get_optimized_params pid=32148)   [repeated 8x across cluster]
(get_optimized_params pid=32148)   [repeated 8x across cluster]
(get_optimized_params pid=32148)   [repeated 8x across cluster]
(get_optimized_params pid=32148)   [repeated 8x across cluster]
(get_optimized_params pid=33200)   [repeated 8x across cluster]
(get_optimized_params pid=33200)   [repeated 8x across cluster]
(get_optimized_params pid=33200)   [repeated 8x across cluster]
(get_optimized_params pid=28184)   [repeated 6x across cluster]
(get_optimized_params pid=33200)   [repeated 2x across cluster]
(get_optimized_params pid=28184)   [repeated 6x across cluster]
(get_optimized_params pid=33200)   [repeated 2x across cluster]
(get_optimized_params pid=28184)   [repeated 6x across cluster]
(get_optimized_params pid=21360)   [repeated 3x across cluster]
(get_optimized_params pid=13420)   [repeated 4x across cluster]
(get_

(get_optimized_params pid=20408)   [repeated 4x across cluster]
(get_optimized_params pid=32148)   [repeated 4x across cluster]
(get_optimized_params pid=28184)   [repeated 5x across cluster]
(get_optimized_params pid=32148)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=32148)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=32148)   [repeated 6x across cluster]
(get_optimized_params pid=28184)   [repeated 2x across cluster]
(get_optimized_params pid=20408)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=20408)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=13420)   [repeated 4x across cluster]
(get_optimized_params pid=33200)   [repeated 6x across cluster]
(get_optimized_params pid=28184)   [repe

(get_optimized_params pid=20408)   [repeated 2x across cluster]
(get_optimized_params pid=32148)   [repeated 7x across cluster]
(get_optimized_params pid=21360)   [repeated 4x across cluster]
(get_optimized_params pid=32148)   [repeated 4x across cluster]
(get_optimized_params pid=33928)   [repeated 3x across cluster]
(get_optimized_params pid=28184)   [repeated 4x across cluster]
(get_optimized_params pid=33928)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=20408)   [repeated 4x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=32148)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=32148)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 3x across cluster]
(get_optimized_params pid=32148)   [repeated 6x across cluster]
(get_optimized_params pid=28184)   [repe

(get_optimized_params pid=20408)   [repeated 4x across cluster]
(get_optimized_params pid=33200)   [repeated 4x across cluster]
(get_optimized_params pid=32148)   [repeated 3x across cluster]
(get_optimized_params pid=21360)   [repeated 5x across cluster]
(get_optimized_params pid=28184)   [repeated 2x across cluster]
(get_optimized_params pid=21360)   [repeated 6x across cluster]
(get_optimized_params pid=32148)   [repeated 2x across cluster]
(get_optimized_params pid=21360)   [repeated 6x across cluster]
(get_optimized_params pid=32148)   [repeated 2x across cluster]
(get_optimized_params pid=21360)   [repeated 6x across cluster]
(get_optimized_params pid=32148)   [repeated 2x across cluster]
(get_optimized_params pid=28052)   [repeated 8x across cluster]
(get_optimized_params pid=28052)   [repeated 8x across cluster]
(get_optimized_params pid=28184)   [repeated 6x across cluster]
(get_optimized_params pid=13420)   [repeated 2x across cluster]
(get_optimized_params pid=20408)   [repe

(get_optimized_params pid=33200)   [repeated 4x across cluster]
(get_optimized_params pid=33928)   [repeated 4x across cluster]
(get_optimized_params pid=21360)   [repeated 2x across cluster]
(get_optimized_params pid=21360)   [repeated 6x across cluster]
(get_optimized_params pid=20408)   [repeated 5x across cluster]
(get_optimized_params pid=13420)   [repeated 5x across cluster]
(get_optimized_params pid=20408)   [repeated 5x across cluster]
(get_optimized_params pid=28052)   [repeated 2x across cluster]
(get_optimized_params pid=13420)   [repeated 4x across cluster]
(get_optimized_params pid=28052)   [repeated 2x across cluster]
(get_optimized_params pid=33200)   [repeated 4x across cluster]
(get_optimized_params pid=33928)   [repeated 2x across cluster]
(get_optimized_params pid=21360)   [repeated 2x across cluster]
(get_optimized_params pid=33200)   [repeated 3x across cluster]
(get_optimized_params pid=21360)   [repeated 3x across cluster]
(get_optimized_params pid=33928)   [repe

(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_params pid=28052)  
(get_optimized_param

In [84]:
lt/60

61.016666666666666

In [85]:
optimized_params_1 = dict(zip(filtered_sector,[x[0] for x in optimized_params]))
optimized_params_2 = dict(zip(filtered_sector,[x[1] for x in optimized_params]))

In [86]:
result_train = {}
result_test = {}
list_summary = []

for sector_code in filtered_sector:
    
    if len(dict_stk[sector_code]) == 0:
        continue
        
    params = optimized_params_1[sector_code]
    df_train = dict_train[sector_code]
    df_test = dict_test[sector_code]
    
    temp_train = simulation_func(df_train, params)
    temp_test = simulation_func(df_test, params)
    
    ret_train = pd.DataFrame([temp_train[1], temp_train[2], temp_train[3],temp_train[4],temp_train[5]], 
                             index = ['init_dt','exit_dt','ls','long','short']).T
    ret_test = pd.DataFrame([temp_test[1], temp_test[2], temp_test[3],temp_test[4],temp_test[5]], 
                            index = ['init_dt','exit_dt','ls','long','short']).T
    
    ret_train['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['init_dt']])
    ret_train['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_train['exit_dt']])
    ret_test['init_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['init_dt']])
    ret_test['exit_dt'] = pd.to_datetime([str(int(x)) for x in ret_test['exit_dt']])
    
    ret_train['holding period'] = ret_train['exit_dt'] - ret_train['init_dt']
    ret_test['holding period'] = ret_test['exit_dt'] - ret_test['init_dt']
    
    result_train[sector_code] = ret_train
    result_test[sector_code] = ret_test
    
    
    case_train = len(ret_train)
    
    if case_train == 0:
        hr_train = 0
    else:
        hr_train = len(ret_train[ret_train['ls'] >= 0])/len(ret_train)
        
    cumret_train = temp_train[0]
    
    holding_period_train = ret_train['holding period'].median().days
    
    case_test = len(ret_test)
    
    if case_test == 0:
        hr_test = 0
    else:
        hr_test = len(ret_test[ret_test['ls'] >= 0])/len(ret_test)
    
    cumret_test = temp_test[0]
    
    holding_period_test = ret_test['holding period'].median().days
                   
    
    list_summary.append([case_train, hr_train, cumret_train, holding_period_train, case_test, hr_test, cumret_test, holding_period_test])
    

In [87]:
df_summary = pd.DataFrame(list_summary, columns = ['cnt_train','hr_train','ret_train','hp_train','cnt_test','hr_test','ret_test','hp_test'], 
                          index = [dict_sector_name[x] for x in result_train.keys()])
df_summary['keys'] = result_train.keys()

In [88]:
df_summary

,cnt_train,hr_train,ret_train,hp_train,cnt_test,hr_test,ret_test,hp_test,keys
음식료품,5,0.800000,3.791771,78,1,0.000000,0.620993,878.0,IKS005
섬유의복,3,0.666667,-3.431120,149,0,0.000000,1.000000,NaN,IKS006
화학,3,1.000000,2.513162,173,1,1.000000,1.151542,199.0,IKS008
의약품,3,1.000000,6.471656,494,0,0.000000,1.000000,NaN,IKS009
비금속광물,11,0.818182,0.273574,108,4,0.750000,1.258471,172.0,IKS010
철강금속,8,1.000000,4.863630,154,1,0.000000,0.752333,455.0,IKS011
기계,13,1.000000,19.780397,107,4,0.750000,-2.311550,201.0,IKS012
전기전자,3,0.666667,1.692177,535,1,1.000000,1.697570,266.0,IKS013
운수장비,5,1.000000,6.512855,206,0,0.000000,1.000000,NaN,IKS015
유통업,8,0.750000,-0.263448,187,1,1.000000,1.025647,415.0,IKS016


In [65]:
result_train['IKS013']

,init_dt,exit_dt,ls,long,short,holding period
0,2010-07-09,2011-01-18,0.119234,0.083057,0.036177,193 days
1,2011-06-17,2011-09-28,0.138036,-0.157386,0.295423,103 days
2,2011-12-28,2012-04-03,0.305289,0.353550,-0.048261,97 days
3,2012-06-22,2013-03-26,0.210298,0.172644,0.037654,277 days
4,2013-07-29,2014-02-11,0.352197,0.270089,0.082109,197 days
5,2014-04-14,2014-06-05,0.197675,0.165654,0.032020,52 days
6,2014-09-03,2014-12-09,0.151036,0.123182,0.027854,97 days
7,2015-03-31,2015-05-04,0.119886,0.028355,0.091531,34 days
8,2015-10-01,2017-01-11,0.579343,0.507310,0.072033,468 days
9,2017-03-17,2017-07-11,0.176679,0.361395,-0.184716,116 days


In [66]:
result_test['IKS013']

,init_dt,exit_dt,ls,long,short,holding period
0,2020-07-02,2020-11-17,0.115723,0.197789,-0.082066,138 days
1,2021-05-31,2022-02-10,0.111159,0.021912,0.089247,255 days


In [89]:
ray.shutdown()